<a href="https://colab.research.google.com/github/Winindu/ML_CW/blob/master/Copy_of_telco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, random
import numpy as np
import pandas as pd

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Telco Customers.csv")

df.drop_duplicates(inplace=True)
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"].replace(" ", np.nan), errors="coerce")
df["TotalCharges"].fillna(df["MonthlyCharges"] * df["tenure"], inplace=True)

if "customerID" in df.columns:
    df.drop(columns=["customerID"], inplace=True)

print(df.head())
print(df.shape)


In [ ]:
print("\n=== Dataset Shape ===")
print(df.shape)

print("\n=== First 5 Rows ===")
display(df.head())

print("\n=== Data Types ===")
print(df.dtypes)

print("\n=== Summary Statistics (Numerical) ===")
display(df.describe())

print("\n=== Summary Statistics (Categorical) ===")
display(df.describe(include=['object']))


In [ ]:
plt.figure(figsize=(10,4))
sns.heatmap(df.isnull(), cbar=False, cmap="viridis")
plt.title("Missing Values Heatmap")
plt.show()


In [ ]:
plt.figure(figsize=(5,3))
sns.countplot(data=df, x="Churn", palette="coolwarm")
plt.title("Churn Distribution")
plt.show()


In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols].hist(figsize=(12,8), bins=30)
plt.suptitle("Numerical Feature Distributions", y=1.02)
plt.show()


In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df[num_cols].corr(), annot=False, cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.show()


In [ ]:
for col in num_cols:
    plt.figure(figsize=(6,4))
    sns.boxplot(data=df, x="Churn", y=col)
    plt.title(f"{col} vs Churn")
    plt.show()


In [ ]:
y = df["Churn"].map({"No":0, "Yes":1})
X = df.drop(columns=["Churn"])

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()

try:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
except:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

preprocessor = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols),

    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", ohe)
    ]), cat_cols)
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

preprocessor.fit(X_train)
X_train_prep = preprocessor.transform(X_train)
X_test_prep = preprocessor.transform(X_test)


In [ ]:
sm = SMOTE(random_state=SEED)
X_train_bal, y_train_bal = sm.fit_resample(X_train_prep, y_train)
print("Balanced classes:", np.bincount(y_train_bal))
